<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Лишние-символы" data-toc-modified-id="Лишние-символы-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Лишние символы</a></span></li><li><span><a href="#Токенизация" data-toc-modified-id="Токенизация-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Токенизация</a></span></li><li><span><a href="#Стоп-слова" data-toc-modified-id="Стоп-слова-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Стоп слова</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Выделение-признаков-и-разбиение-данных" data-toc-modified-id="Выделение-признаков-и-разбиение-данных-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Выделение признаков и разбиение данных</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#MultinomialNB" data-toc-modified-id="MultinomialNB-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>MultinomialNB</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение тональности текста

# Входные данные
Даны тексты с их окраской (токсичные/нейтральные)

# Задача
Создать модель, которая будет определять окраску текста

Подключим необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import re
import torch
import transformers
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from tqdm import notebook

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
pd.set_option('display.max_colwidth', -1)
warnings.filterwarnings("ignore")

In [4]:
SEED = 63

## Подготовка

### Загрузка данных

Загрузим данные и посмотрим на 5 случайных записей

In [5]:
df = pd.read_csv('~/Documents/datasets/toxic_comments.csv')
df.sample(n=5, random_state=SEED)

,text,toxic
77238,you're wasting your time biatch you know i'll be back again and again and again with a different IP adress,1
26002,"And besides, there isn't a page for Fictional people who wear shoes. Or Fictional confectioners for that matter. Not that I know of anyway. And I'll have you know the Snow Queen wasn't the only influence for the White Witch. Jadis was also largely based on Ayesha, hardly a Wicked Witch of the West persona.",0
24683,"""\n\n Edits \n\nHello, thank you for letting me know about your changes, I briefly looked over the article I changed (technically changed back) and was unaware of any errors, The reason I made the changes was due to me looking at another article, and noticing the user named """"Lukeno94"""" making an incorrect edit, so I found my way to his page of edits, and it seemed he was targeting a specific IP address and changing anything that person edited. I apologize if I made an error, I was simply trying to help out what looked like someone being bullied.""",0
116916,BTW: reverting things back without first having a related discussion is seen as unfriendly act.,0
116920,"Blocking this IP \n\nhi,\n\nJust to let you know, this IP is from a public terminal at Walter Library on the University of Minnesota campus, so I imagine blocking it won't do much good in stopping the abuser you are after.",0


Нам будет необходимо очистить тексты от ненужных символов и чисел (они нам не помогут). После чего мы применим токенизацию - разделим тексты на отдельные слова, а затем лемматизацию - приведение слов к их словарным формам (леммам)<br>

### Анализ данных

Приступим у анализу данных, посмотрим, с чем мы будет работать

In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 80.1 MB


В датасете 159571 запись. Пропусков нет

In [7]:
print('Дубликатов:', df.duplicated().sum())

Дубликатов: 0


Дубликатов нет. Посмотрим на количество данных разного класса

In [8]:
value_count = df['toxic'].value_counts().to_frame()
value_count.columns = ['Количество записей']
value_count['Процент'] = (value_count['Количество записей'] / df.shape[0] * 100).round(1)
value_count['Окраска текста'] = ['Нейтральный', 'Токсичный']
value_count

,Количество записей,Процент,Окраска текста
0,143346,89.8,Нейтральный
1,16225,10.2,Токсичный


В данных 10% токсичных записей и 90% нейтральных

### Подготовка данных

#### Лишние символы

Для начала, избавимся от лишнего в тексте. Заодно приведём весь текст к нижнему регистру

In [9]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z ]', ' ', text)

In [10]:
df['cleared'] = df['text'].apply(lambda x:clean_text(x.lower()))
df.sample(n=5, random_state=SEED)

,text,toxic,cleared
77238,you're wasting your time biatch you know i'll be back again and again and again with a different IP adress,1,you re wasting your time biatch you know i ll be back again and again and again with a different ip adress
26002,"And besides, there isn't a page for Fictional people who wear shoes. Or Fictional confectioners for that matter. Not that I know of anyway. And I'll have you know the Snow Queen wasn't the only influence for the White Witch. Jadis was also largely based on Ayesha, hardly a Wicked Witch of the West persona.",0,and besides there isn t a page for fictional people who wear shoes or fictional confectioners for that matter not that i know of anyway and i ll have you know the snow queen wasn t the only influence for the white witch jadis was also largely based on ayesha hardly a wicked witch of the west persona
24683,"""\n\n Edits \n\nHello, thank you for letting me know about your changes, I briefly looked over the article I changed (technically changed back) and was unaware of any errors, The reason I made the changes was due to me looking at another article, and noticing the user named """"Lukeno94"""" making an incorrect edit, so I found my way to his page of edits, and it seemed he was targeting a specific IP address and changing anything that person edited. I apologize if I made an error, I was simply trying to help out what looked like someone being bullied.""",0,edits hello thank you for letting me know about your changes i briefly looked over the article i changed technically changed back and was unaware of any errors the reason i made the changes was due to me looking at another article and noticing the user named lukeno making an incorrect edit so i found my way to his page of edits and it seemed he was targeting a specific ip address and changing anything that person edited i apologize if i made an error i was simply trying to help out what looked like someone being bullied
116916,BTW: reverting things back without first having a related discussion is seen as unfriendly act.,0,btw reverting things back without first having a related discussion is seen as unfriendly act
116920,"Blocking this IP \n\nhi,\n\nJust to let you know, this IP is from a public terminal at Walter Library on the University of Minnesota campus, so I imagine blocking it won't do much good in stopping the abuser you are after.",0,blocking this ip hi just to let you know this ip is from a public terminal at walter library on the university of minnesota campus so i imagine blocking it won t do much good in stopping the abuser you are after


#### Токенизация

Разобьём тексты на слова

In [11]:
def split_text(text):
    return text.split()

In [12]:
df['splited'] = df['cleared'].apply(lambda x:split_text(x))
df.sample(n=5, random_state=SEED)

,text,toxic,cleared,splited
77238,you're wasting your time biatch you know i'll be back again and again and again with a different IP adress,1,you re wasting your time biatch you know i ll be back again and again and again with a different ip adress,"[you, re, wasting, your, time, biatch, you, know, i, ll, be, back, again, and, again, and, again, with, a, different, ip, adress]"
26002,"And besides, there isn't a page for Fictional people who wear shoes. Or Fictional confectioners for that matter. Not that I know of anyway. And I'll have you know the Snow Queen wasn't the only influence for the White Witch. Jadis was also largely based on Ayesha, hardly a Wicked Witch of the West persona.",0,and besides there isn t a page for fictional people who wear shoes or fictional confectioners for that matter not that i know of anyway and i ll have you know the snow queen wasn t the only influence for the white witch jadis was also largely based on ayesha hardly a wicked witch of the west persona,"[and, besides, there, isn, t, a, page, for, fictional, people, who, wear, shoes, or, fictional, confectioners, for, that, matter, not, that, i, know, of, anyway, and, i, ll, have, you, know, the, snow, queen, wasn, t, the, only, influence, for, the, white, witch, jadis, was, also, largely, based, on, ayesha, hardly, a, wicked, witch, of, the, west, persona]"
24683,"""\n\n Edits \n\nHello, thank you for letting me know about your changes, I briefly looked over the article I changed (technically changed back) and was unaware of any errors, The reason I made the changes was due to me looking at another article, and noticing the user named """"Lukeno94"""" making an incorrect edit, so I found my way to his page of edits, and it seemed he was targeting a specific IP address and changing anything that person edited. I apologize if I made an error, I was simply trying to help out what looked like someone being bullied.""",0,edits hello thank you for letting me know about your changes i briefly looked over the article i changed technically changed back and was unaware of any errors the reason i made the changes was due to me looking at another article and noticing the user named lukeno making an incorrect edit so i found my way to his page of edits and it seemed he was targeting a specific ip address and changing anything that person edited i apologize if i made an error i was simply trying to help out what looked like someone being bullied,"[edits, hello, thank, you, for, letting, me, know, about, your, changes, i, briefly, looked, over, the, article, i, changed, technically, changed, back, and, was, unaware, of, any, errors, the, reason, i, made, the, changes, was, due, to, me, looking, at, another, article, and, noticing, the, user, named, lukeno, making, an, incorrect, edit, so, i, found, my, way, to, his, page, of, edits, and, it, seemed, he, was, targeting, a, specific, ip, address, and, changing, anything, that, person, edited, i, apologize, if, i, made, an, error, i, was, simply, trying, to, help, out, what, looked, like, someone, being, bullied]"
116916,BTW: reverting things back without first having a related discussion is seen as unfriendly act.,0,btw reverting things back without first having a related discussion is seen as unfriendly act,"[btw, reverting, things, back, without, first, having, a, related, discussion, is, seen, as, unfriendly, act]"
116920,"Blocking this IP \n\nhi,\n\nJust to let you know, this IP is from a public terminal at Walter Library on the University of Minnesota campus, so I imagine blocking it won't do much good in stopping the abuser you are after.",0,blocking this ip hi just to let you know this ip is from a public terminal at walter library on the university of minnesota campus so i imagine blocking it won t do much good in stopping the abuser you are after,"[blocking, this, ip, hi, just, to, let, you, know, this, ip, is, from, a, public, terminal, at, walter, library, on, the, university, of, minnesota, campus,

#### Стоп слова

Уберём "стоп слова"

In [13]:
stop_words = set(stopwords.words('english')) 

In [14]:
def remove_stop_words(text):
     return [word for word in text if word not in stop_words]

In [15]:
df['no_stop_words'] = df['splited'].apply(lambda x:remove_stop_words(x))
df.sample(n=5, random_state=SEED)

,text,toxic,cleared,splited,no_stop_words
77238,you're wasting your time biatch you know i'll be back again and again and again with a different IP adress,1,you re wasting your time biatch you know i ll be back again and again and again with a different ip adress,"[you, re, wasting, your, time, biatch, you, know, i, ll, be, back, again, and, again, and, again, with, a, different, ip, adress]","[wasting, time, biatch, know, back, different, ip, adress]"
26002,"And besides, there isn't a page for Fictional people who wear shoes. Or Fictional confectioners for that matter. Not that I know of anyway. And I'll have you know the Snow Queen wasn't the only influence for the White Witch. Jadis was also largely based on Ayesha, hardly a Wicked Witch of the West persona.",0,and besides there isn t a page for fictional people who wear shoes or fictional confectioners for that matter not that i know of anyway and i ll have you know the snow queen wasn t the only influence for the white witch jadis was also largely based on ayesha hardly a wicked witch of the west persona,"[and, besides, there, isn, t, a, page, for, fictional, people, who, wear, shoes, or, fictional, confectioners, for, that, matter, not, that, i, know, of, anyway, and, i, ll, have, you, know, the, snow, queen, wasn, t, the, only, influence, for, the, white, witch, jadis, was, also, largely, based, on, ayesha, hardly, a, wicked, witch, of, the, west, persona]","[besides, page, fictional, people, wear, shoes, fictional, confectioners, matter, know, anyway, know, snow, queen, influence, white, witch, jadis, also, largely, based, ayesha, hardly, wicked, witch, west, persona]"
24683,"""\n\n Edits \n\nHello, thank you for letting me know about your changes, I briefly looked over the article I changed (technically changed back) and was unaware of any errors, The reason I made the changes was due to me looking at another article, and noticing the user named """"Lukeno94"""" making an incorrect edit, so I found my way to his page of edits, and it seemed he was targeting a specific IP address and changing anything that person edited. I apologize if I made an error, I was simply trying to help out what looked like someone being bullied.""",0,edits hello thank you for letting me know about your changes i briefly looked over the article i changed technically changed back and was unaware of any errors the reason i made the changes was due to me looking at another article and noticing the user named lukeno making an incorrect edit so i found my way to his page of edits and it seemed he was targeting a specific ip address and changing anything that person edited i apologize if i made an error i was simply trying to help out what looked like someone being bullied,"[edits, hello, thank, you, for, letting, me, know, about, your, changes, i, briefly, looked, over, the, article, i, changed, technically, changed, back, and, was, unaware, of, any, errors, the, reason, i, made, the, changes, was, due, to, me, looking, at, another, article, and, noticing, the, user, named, lukeno, making, an, incorrect, edit, so, i, found, my, way, to, his, page, of, edits, and, it, seemed, he, was, targeting, a, specific, ip, address, and, changing, anything, that, person, edited, i, apologize, if, i, made, an, error, i, was, simply, trying, to, help, out, what, looked, like, someone, being, bullied]","[edits, hello, thank, letting, know, changes, briefly, looked, article, changed, technically, changed, back, unaware, errors, reason, made, changes, due, looking, another, article, noticing, user, named, lukeno, making, incorrect, edit, found, way, page, edits, seemed, targeting, specific, ip, address, changing, anything, person, edited, apologize, made, error, simply, trying, help, looked, like, someone, bullied]"
116916,BTW: reverting things back without first having a related discussion is seen as unfriendly act.,0,btw reverting things back without first having a related discussion is seen as unfriendly act,"[b

#### Лемматизация

Теперь проведём лемматизацию

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
def lemmatize(text):
    return [lemmatizer.lemmatize(word) for word in text]

In [18]:
df['lemmas'] = df['no_stop_words'].apply(lambda x:lemmatize(x))
df.sample(n=5, random_state=SEED)

,text,toxic,cleared,splited,no_stop_words,lemmas
77238,you're wasting your time biatch you know i'll be back again and again and again with a different IP adress,1,you re wasting your time biatch you know i ll be back again and again and again with a different ip adress,"[you, re, wasting, your, time, biatch, you, know, i, ll, be, back, again, and, again, and, again, with, a, different, ip, adress]","[wasting, time, biatch, know, back, different, ip, adress]","[wasting, time, biatch, know, back, different, ip, adress]"
26002,"And besides, there isn't a page for Fictional people who wear shoes. Or Fictional confectioners for that matter. Not that I know of anyway. And I'll have you know the Snow Queen wasn't the only influence for the White Witch. Jadis was also largely based on Ayesha, hardly a Wicked Witch of the West persona.",0,and besides there isn t a page for fictional people who wear shoes or fictional confectioners for that matter not that i know of anyway and i ll have you know the snow queen wasn t the only influence for the white witch jadis was also largely based on ayesha hardly a wicked witch of the west persona,"[and, besides, there, isn, t, a, page, for, fictional, people, who, wear, shoes, or, fictional, confectioners, for, that, matter, not, that, i, know, of, anyway, and, i, ll, have, you, know, the, snow, queen, wasn, t, the, only, influence, for, the, white, witch, jadis, was, also, largely, based, on, ayesha, hardly, a, wicked, witch, of, the, west, persona]","[besides, page, fictional, people, wear, shoes, fictional, confectioners, matter, know, anyway, know, snow, queen, influence, white, witch, jadis, also, largely, based, ayesha, hardly, wicked, witch, west, persona]","[besides, page, fictional, people, wear, shoe, fictional, confectioner, matter, know, anyway, know, snow, queen, influence, white, witch, jadis, also, largely, based, ayesha, hardly, wicked, witch, west, persona]"
24683,"""\n\n Edits \n\nHello, thank you for letting me know about your changes, I briefly looked over the article I changed (technically changed back) and was unaware of any errors, The reason I made the changes was due to me looking at another article, and noticing the user named """"Lukeno94"""" making an incorrect edit, so I found my way to his page of edits, and it seemed he was targeting a specific IP address and changing anything that person edited. I apologize if I made an error, I was simply trying to help out what looked like someone being bullied.""",0,edits hello thank you for letting me know about your changes i briefly looked over the article i changed technically changed back and was unaware of any errors the reason i made the changes was due to me looking at another article and noticing the user named lukeno making an incorrect edit so i found my way to his page of edits and it seemed he was targeting a specific ip address and changing anything that person edited i apologize if i made an error i was simply trying to help out what looked like someone being bullied,"[edits, hello, thank, you, for, letting, me, know, about, your, changes, i, briefly, looked, over, the, article, i, changed, technically, changed, back, and, was, unaware, of, any, errors, the, reason, i, made, the, changes, was, due, to, me, looking, at, another, article, and, noticing, the, user, named, lukeno, making, an, incorrect, edit, so, i, found, my, way, to, his, page, of, edits, and, it, seemed, he, was, targeting, a, specific, ip, address, and, changing, anything, that, person, edited, i, apologize, if, i, made, an, error, i, was, simply, trying, to, help, out, what, looked, like, someone, being, bullied]","[edits, hello, thank, letting, know, changes, briefly, looked, article, changed, technically, changed, back, unaware, errors, reason, made, changes, due, looking, another, article, noticing, user, named, lukeno, making, incorrect, edit, found, way, page, edits, seemed, targeting, specific, ip, address, changing, anything, person, edited

#### Выделение признаков и разбиение данных

Выделим признаки, целевой признак и разделим на обучающую и тестовую выборки

In [19]:
X = df['lemmas']
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)
print('Количество элементов обучающей выборки: {:.0f}%'.format(X_train.shape[0]/df.shape[0] * 100))
print('Количество элементов тестовой выборки: {:.0f}%'.format(X_test.shape[0]/df.shape[0] * 100))

Количество элементов обучающей выборки: 80%
Количество элементов тестовой выборки: 20%


Данные разбиты верно<br>
Вычислим tf-idf для текстов

In [20]:
X_train = X_train.astype('U')
X_test = X_test.astype('U')
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
X_train_vec = count_tf_idf.fit_transform(X_train)
X_test_vec = count_tf_idf.transform(X_test)

In [21]:
print('Размер обучающей выборки:', X_train_vec.shape)
print('Размер тестовой выборки:', X_test_vec.shape)

Размер обучающей выборки: (127656, 139229)
Размер тестовой выборки: (31915, 139229)


### Вывод

Мы загрузили данные, провели краткий анализ и подготовили данные к работе с моделями машинного обучения

## Обучение

Данные подготовлены, приступаем к проверке моделей машинного обучения. Использовать будем следующие модели:
- логистическая регрессия
- дерево решений
- случайый лес
- MultinomialNB (документация sklearn предлагает этот классификатор для работы с текстом https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#training-a-classifier)

### Логистическая регрессия

Обучим линейную регрессию и проверим её на обучающей и тестовой выборках

In [22]:
model = LogisticRegression(random_state=SEED, class_weight='balanced') 
model.fit(X_train_vec, y_train)
predictions = model.predict(X_train_vec)
f1 = f1_score(y_train, predictions)
print('F1 на обучающей выборке:', f1)

predictions = model.predict(X_test_vec)
f1 = f1_score(y_test, predictions)
print('F1 на тестовой выборке:', f1)

F1 на обучающей выборке: 0.8305169483051694
F1 на тестовой выборке: 0.7519806633543709


Модель проходит по требованию к метрике F1 - не меньше 0.75

### Дерево решений

Обучим дерево решений с разным количеством значения максимальной глубины

In [23]:
depth = 1
max_depth = 25
for i in notebook.tqdm(range(max_depth)):
    model = DecisionTreeClassifier(max_depth=depth, 
                                   random_state=SEED, 
                                   class_weight='balanced')
    model.fit(X_train_vec, y_train)
    predictions = model.predict(X_test_vec)
    f1 = f1_score(y_test, predictions)
    print('F1 на тестовой выборке:', f1, 'при глубине:', depth)
    
    depth = depth + 1

F1 на тестовой выборке: 0.2654028436018957 при глубине: 1
F1 на тестовой выборке: 0.365075439030423 при глубине: 2
F1 на тестовой выборке: 0.36386138613861385 при глубине: 3
F1 на тестовой выборке: 0.422475106685633 при глубине: 4
F1 на тестовой выборке: 0.42796809009854536 при глубине: 5
F1 на тестовой выборке: 0.47280145058930184 при глубине: 6
F1 на тестовой выборке: 0.5045592705167173 при глубине: 7
F1 на тестовой выборке: 0.5357519510651761 при глубине: 8
F1 на тестовой выборке: 0.5539613225202745 при глубине: 9
F1 на тестовой выборке: 0.5664904432696218 при глубине: 10
F1 на тестовой выборке: 0.5782312925170068 при глубине: 11
F1 на тестовой выборке: 0.5838847800700662 при глубине: 12
F1 на тестовой выборке: 0.5938631599008958 при глубине: 13
F1 на тестовой выборке: 0.5950920245398772 при глубине: 14
F1 на тестовой выборке: 0.6072106261859583 при глубине: 15
F1 на тестовой выборке: 0.597860962566845 при глубине: 16
F1 на тестовой выборке: 0.6011883956658511 при глубине: 17
F1 на 

Дерево решений с задачей справляется хуже. Наилучшая F1 0.61

### Случайный лес

In [24]:
depth = 1
est = 10
max_depth = 15
max_estimators = 50
estimators_step = 10
best_est = 0
best_depth = 0
best_f1 = 0
iterations = int(max_depth * (max_estimators/estimators_step))
for i in notebook.tqdm(range(iterations)):
    model = RandomForestClassifier(max_depth=depth, 
                                   n_estimators=est,
                                   random_state=SEED, 
                                   class_weight='balanced')
    model.fit(X_train_vec, y_train)
    predictions = model.predict(X_test_vec)
    f1 = f1_score(y_test, predictions)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth = depth
        best_est = est
    
    if (est != max_estimators):
        est = est + 10
    else:
        est = 10
        depth = depth + 1

print('Лучшая метрика F1:', best_f1)
print('Глубина:', best_depth)
print('Количество оценщиков', best_est)


Лучшая метрика F1: 0.5267774086378738
Глубина: 4
Количество оценщиков 40


Случайный лес также не справился с задачей

### MultinomialNB

In [25]:
model = MultinomialNB(alpha=0.01) 
model.fit(X_train_vec, y_train)
predictions = model.predict(X_test_vec)
f1 = f1_score(y_test, predictions)
print('F1 на тестовой выборке:', f1)

F1 на тестовой выборке: 0.6709265175718849


MultinomialNB не справился с задачей

### Выводы

Единственной моделью, которая справилась с поставленной задачей - Логистическая регрессия, показав метрику F1 0.751<br>

## Выводы

1) В нашем распоряжении оказались тексты с их окраской: нейтральный или токсичный текст<br>

2) Убедились, что данные без пропусков и дубликатов<br>

3) Мы преобразовали тексты: <br>
    - убрали лишние слова <br>
    - токенизировали <br>
    - убрали стоп-слова <br>
    - провели лемматизацию <br>
    - выделили признаки <br>
    - разбили данные <br>
    
4) Обучили несколько моделей машинного обучения и проверили на текстах <br>

5) Единственной моделью, которая справилась с поставленной задачей (F1 метрика не ниже 0.75) оказалась Линейная регрессия, которая показала значение метрики F1 = 0.751 <br>